In [1]:
import torch

from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import DataLoader, random_split

In [2]:
import itertools

In [3]:
from helpers import CVFConfigForGCNDataset

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [6]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
dataset_6b = CVFConfigForGCNDataset(
    device,
    "graph_6b_config_rank_dataset.csv",
    "graph_6b_edge_index.json",
    20,
)

dataset_n5 = CVFConfigForGCNDataset(
    device,
    "graph_5_config_rank_dataset.csv",
    "graph_5_edge_index.json",
    20
)

dataset_pl_n5 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n5_edge_index.json",
    20
)

dataset_pl_n6 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n6_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n6_edge_index.json",
    20
)

dataset_pl_n7 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n7_edge_index.json",
    20
)

dataset_pl_n8 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n8_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n8_edge_index.json",
    20
)

dataset_pl_n9 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n9_edge_index.json",
    20,
)
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )

batch_size = 64

# dataset_coll = [dataset_pl_n5, dataset_pl_n6, dataset_pl_n7, dataset_pl_n8]
dataset_coll = [dataset_pl_n9]
train_dataloader_coll = []
test_dataloader_coll = []

for dataset in dataset_coll:
    train_size = int(0.75 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    train_dataloader_coll.append(train_loader)
    test_dataloader_coll.append(test_loader)

train_dataloader_coll_iter = [iter(i) for i in train_dataloader_coll]

In [7]:
def generate_batch():
    end_loop = [False for _ in range(len(train_dataloader_coll))]
    while not all(end_loop):
        for di, data_loader in enumerate(train_dataloader_coll_iter):
            if end_loop[di]:
                continue
            try:
                batch = next(data_loader)
            except StopIteration:
                end_loop[di] = True
                continue
            yield batch, di

In [8]:
# cnt = 0
# for i in generate_batch():
#     cnt += 1

# print(cnt)
print("Number of batches:", [len(i) for i in train_dataloader_coll])

Number of batches: [16200]


In [9]:
def r2_score(y_true, y_pred):
    """
    Calculate R-squared (coefficient of determination) for regression in PyTorch.
    
    Args:
        y_true (torch.Tensor): True target values.
        y_pred (torch.Tensor): Predicted values.
        
    Returns:
        float: R-squared value.
    """
    # Ensure that the inputs are float tensors for accurate computation
    y_true = y_true.float()
    y_pred = y_pred.float()
    
    # Calculate the total sum of squares (TSS)
    total_sum_of_squares = torch.sum((y_true - torch.mean(y_true)) ** 2)
    
    # Calculate the residual sum of squares (RSS)
    residual_sum_of_squares = torch.sum((y_true - y_pred) ** 2)
    
    # Calculate R-squared
    r2 = 1 - (residual_sum_of_squares / total_sum_of_squares)
    
    return r2.item()  # Return as a Python float

In [10]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = torch.relu(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, epochs):
        # criterion = torch.nn.CrossEntrop yLoss()
        # criterion = torch.nn.MSELoss()
        criterion_sum = torch.nn.MSELoss(reduction='sum')
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        # edge_index = dataset.edge_index.t().to(device)
        dataloaders = itertools.tee(generate_batch(), epochs)
        for epoch in range(1, epochs + 1):
            max_loss = -1
            total_loss = 0
            count = 0
            for batch, di in dataloaders[epoch-1]:
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, dataset_coll[di].edge_index)
                # print("output", out, "y", y)
                loss = criterion_sum(out, y)
                # loss_sum = cr(out, y)
                avg_loss = loss/len(batch[1])
                if avg_loss > max_loss:
                    max_loss = avg_loss
                total_loss += loss
                count += len(batch[1])
                loss.backward()
                optimizer.step()

            if count > 0:
                print("Epoch", epoch, "| Loss:", total_loss / count, "| Max Loss:", max_loss)


In [11]:
gnn = VanillaGNN(20, 64, 1).to(device)
print(gnn)

gnn.fit(epochs=10)

VanillaGNN(
  (gcn1): GCNConv(20, 64)
  (gcn2): GCNConv(64, 64)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


Epoch 1 | Loss: tensor(0.3745, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(6.9730, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 | Loss: tensor(0.2516, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.5754, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 | Loss: tensor(0.2202, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.4540, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 | Loss: tensor(0.2032, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.4197, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 | Loss: tensor(0.1916, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.4059, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 | Loss: tensor(0.1839, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.3865, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 | Loss: tensor(0.1810, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(0.3805, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 8 | Loss: tens

In [13]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

criterion = torch.nn.MSELoss(reduction="sum")
total_loss = 0

total_matched = 0

# indx = 1
# dataset = dataset_coll[indx]
# test_loader = test_dataloader_coll[indx]
# test_dataset = test_loader.dataset

dataset = dataset_pl_n7
train_size = int(0.5 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

r_acc = 0

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, dataset.edge_index)
    # print(y.squeeze(-1).squeeze(-1)[:100], predicted.squeeze(-1).squeeze(-1)[:100])
    r_acc = r2_score(y, predicted)
    loss = criterion(predicted, y)
    total_loss += loss
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    # print(y.squeeze(-1).squeeze(-1), predicted.squeeze(-1).squeeze(-1))

# print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")
print("Total matched", total_matched, "out of", len(test_dataset))
# print("Total Avg. Loss", total_loss/len(test_loader.dataset))
print("R-squared accuracy", round(r_acc * 100, 3), "%")

Total matched 3724 out of 13500
R-squared accuracy 26.862 %
